# The ROOT file

* With ROOT, objects can be written to files

* ROOT provides its own file class, [TFile](https://root.cern/doc/master/classTFile.html), to interact with these files

* ROOT files are _binary_ and can be transparently _compressed_ to reduce disk usage

* ROOT files have a logical “file-system-like” structure

  * E.g. a directory hierarchy


This is how you create a `TFile`:

In [ ]:
import ROOT

f = ROOT.TFile("my_file.root", "RECREATE")

<img src="img/tfile1.png">

and how you close it (note that when `f` is destroyed, the file is closed automatically):

In [ ]:
f.Close()

- The file we've just created is empty, let's actually write something in it this time.

- We will write a histogram object in it. 

- Note how we create the histogram after creating the file, we write the histogram and we finally close the file.

In [ ]:
f = ROOT.TFile("my_file.root", "RECREATE")
hist = ROOT.TH1F("my_histo", "Example histogram", 100, -4, 4)

f.cd()
hist.Write()
f.Close()

# Read a file

Let's open a file `data/hsimple.root` and plot histograms from it


In [ ]:
inputfile = ROOT.TFile("../../data/hsimple.root")
inputfile.ls()

Add `%jsroot on` for pretty and interactive drawing 

In [ ]:
%jsroot on

In [ ]:
# Get the histograms and the profile from the file
hpx    = inputfile.Get("hpx")
hpxpy  = inputfile.Get("hpxpy")
hprof  = inputfile.Get("hprof")

# Create a canvas and divide it
can = ROOT.TCanvas("can", "A Simple Graph Example", 1200, 500)
can.Divide(3, 1)  # 3 columns, 1 row

can.cd(1)
hpx.Draw()

can.cd(2)
hpxpy.Draw("colz")

can.cd(3)
hprof.Draw()

can.Draw()

# Trees


## The HEP dataset

High Energy Physics data is made of many statistically independent collision events. 

Laying data into an "event class", then serialise and write out `N` instances of the class into a file would be very inefficient. 

In ROOT, a dataset is organised columns that can store elements of any C++ type:
* fundamental types: `int`, `float`
* C++ standard collections: `std::vector`, `std::map`
* User created C++ classes

The ROOT dataset is represented by the `TTree` class and is often simply called a tree. Columns in the dataset are instances of the `TBranch` class (often referred to as "branches").

<img src="img/dataset.png">

- A `TTree` dataset can be written to a `TFile` (just like any other C++ object). 

- The ROOT format is logically and physically (on disk) a columnar format. 

- Different columns can be read from disk independently. 

- This translates into faster IO performance with respect to other dataset formats (HDF5, SQL).

In [ ]:
tree = inputfile.Get("ntuple")
print("Number of entries in the tree:", tree.GetEntries())

tree.Show(0)
tree.Show(20)

There are many ways to access the data in the tree

### 1. Using the TTree::Draw method (fast check distributions)

In [ ]:
can.cd()  # Go back to the main canvas
tree.Draw("px")
can.Draw()

In [ ]:
# if want to specify  a histogram to fill and operate on later
hist1 = ROOT.TH1D("hist1", "", 100, -5, 5)
tree.Draw("px>>hist1")
hist1.Draw()
can.Draw()

We can also apply some cuts on data

In [ ]:
tree.Draw("px", "pz>1")
can.Draw()

Suitable for 2D as well, with draw option on third place, and maximum number of entries on the last place

In [ ]:
hist2 = ROOT.TH2D("hist2", "hist 2;px;pz", 50, -3, 3, 50, 0, 10)
nEntryStop = 2000
tree.Draw("pz:px>>hist2", "", "colz", nEntryStop)
can.Draw()

### 2. Using the TTree::GetEntry method (old method, but everyone is still using it)

When you call tree.GetEntry(i) ROOT reads the i-th event from disk into the tree object and populates the branch variables so you can access them as attributes (e.g. tree.px, tree.py, ...). This is an explicit, event-by-event loop: you control the iteration and any per-event logic.

Pros
- Simple and explicit — very clear what happens per event.
- Fine grained control — easy to implement arbitrary algorithms
- Works with arbitrary Python/C++ code and custom classes.

Cons
- Relatively slow in Python for large datasets because of per-entry Python overhead and many GetEntry calls.
- Manual bookkeeping (looping, histogram filling, activation of branches) is error-prone -- for example we might accidentally access a different index of an array than intended
- Not optimized for columnar IO (unlike RDataFrame or TTree::Draw).
- You should enable only needed branches (SetBranchStatus) to reduce IO overhead.

Use GetEntry for debugging, prototyping, or when you need full procedural control; switch to RDataFrame / TTree::Draw for fast, high-level or large-scale processing.

In [ ]:
nEntries = tree.GetEntries()

for i in range(5):
    tree.GetEntry(i)
    px = tree.px
    py = tree.py
    pz = tree.pz
    print(f"Entry {i} : px={px} py={py} pz={pz}")

In [ ]:
# Filling the same histogram as before using this method
hist3 = ROOT.TH1D("hist3", "", 100, -5, 5)
nEntryStop = 2000
for i in range(nEntryStop):
    tree.GetEntry(i)
    px = tree.px
    hist3.Fill(px)

%jsroot on
hist3.Draw()
can.Draw()


### Now let's make a Tree ourselves

Let it be a kaon tree with its momentum components, energy and the entrynumber

In [ ]:
import ROOT
from array import array

outputFile = ROOT.TFile("mytree.root", "RECREATE")
tree_out = ROOT.TTree("tree", "A simple Tree with simple variables")

# 1-element C-style arrays for scalar branches
px = array('f', [0.0])
py = array('f', [0.0])
pz = array('f', [0.0])
energy = array('f', [0.0])
iEvent = array('i', [0])
mass = 0.495  # mass of the kaon in GeV

tree_out.Branch("iEvent", iEvent, "iEvent/I")
tree_out.Branch("px", px, "px/F")
tree_out.Branch("py", py, "py/F")
tree_out.Branch("pz", pz, "pz/F")
tree_out.Branch("energy", energy, "energy/F")

And then fill the tree with some generated values for px, py, pz and use an energy equation to calculate mass

In [ ]:
nEvents = 1000
r = ROOT.TRandom3(0)

for i in range(nEvents):

    # Set values
    iEvent[0] = i
    px[0] = r.Gaus(0, 1)   # Gaus(mean, sigma)
    py[0] = r.Gaus(0, 1)
    pz[0] = r.Exp(2)       # Exp(lambda)

    # print(f"Event {i}: px={px[0]:.3f}, py={py[0]:.3f}, pz={pz[0]:.3f}")

    e = ROOT.TMath.Sqrt(px[0]**2 + py[0]**2 + pz[0]**2 + mass**2)
    # smear the energy to mimic detector resolution
    energy[0] = r.Gaus(e, 0.005 * e)  # 0.5% resolution

    tree_out.Fill()

The last part is to write our tree (which is currently in RAM) to the file


In [ ]:
outputFile.Write()
outputFile.Close()

# RDataFrames ( The  future of ROOT-based analysis )

Interface is very similar to pandas 

[RDataFrame documentation](https://root.cern/doc/master/classROOT_1_1RDataFrame.html)

- RDF is ROOT's high-level analysis interface. 

- Users define their analysis as a sequence of operations to be performed on the data-frame object; 

    - the framework takes care of the management of the loop over entries as well as low-level details such as I/O and parallelisation.

- RDataFrame provides methods to perform most common operations required by ROOT analyses: 

    - at the same time, users can just as easily specify custom code that will be executed in the event loop.
    
<img src="img/rdf_1.png">

## HEP data analysis with RDataFrame
RDataFrame allows reading and writing trees, aiming at making HEP analysis easy to write and fast to perform.

In [ ]:
import ROOT

treename = "tree"
fileName = "mytree.root"

df = ROOT.RDataFrame(treename, fileName)
print("Dataset contents:")
df.Display().Print()

Now we can `Define` new quantities, `Filter` rows based on custom expressions and retrieve some data aggregations such as a `Count` and a `Mean`:

In [ ]:
definept = df.Define("pt", "TMath::Sqrt(TMath::Power(px,2) + TMath::Power(py,2))")
fil1 = definept.Filter("pz < 2")

print("Number of rows before filter:", df.Count().GetValue())
print("Number of rows after filter:", fil1.Count().GetValue())

print("Mean of column pt:", fil1.Mean("pt").GetValue())

fil1.Display(["px", "py", "pt"]).Print()

In [ ]:
pzcut_hist = (
    df.Filter("pz < 5")
      .Define("mass", "TMath::Sqrt(energy*energy - px*px - py*py - pz*pz)")
      .Histo1D("mass")
)

%jsroot on
pzcut_hist.Draw()
ROOT.gPad.Draw()

In [ ]:
hist2dim = (
    df.Define("mass", "TMath::Sqrt(energy*energy - px*px - py*py - pz*pz)")
      .Histo2D(
          ("hist2dim", "energy vs mass; E, GeV; m, GeV/c^2", 200, 0, 15, 200, 0, 1.5),
          "energy",
          "mass",
      )
)

hist2dim.Draw("colz")
ROOT.gPad.Draw()

# Operation categories in RDataFrame
There are 3 main types of operations you can perform on RDataFrames:

**Transformations**: manipulate the dataset, return a modified RDataFrame for further processing.

| Transformation    | Description                                                |
|-------------------|------------------------------------------------------------|
| Alias()           | Introduce an alias for a particular column name.           |
| Define()          | Creates  a new column in the dataset.                      |
| Filter()          | Filter rows based on user-defined conditions.              |

**Actions**: aggregate (parts of) the dataset into a result.

| Action                        | Description                                                                          |
|------------------------------------|--------------------------------------------------------------------------------------|
| Count()                            | Return the number of events processed.                                               |
| Display()                          | Provides a printable object representing the dataset contents.                       |
| Graph()                            | Fills a TGraph  with the two columns provided.                                       |
| Histo1D(), Histo2D(), Histo3D()    | Fill a one-, two-, three-dimensional histogram with the processed column values.     |
| Max(), Min()                       | Return the maximum(minimum) of processed column values.                              |
| Snapshot()        | Writes processed data-set to a new TTree.              |
| ...                                | ...  

**Queries**: these methods  query information about your dataset and the RDataFrame status.

| Operation           | Description                                                                              |
|---------------------|------------------------------------------------------------------------------------------|
| GetColumnNames()    | Get the names of all the available columns of the dataset.                               |
| GetColumnType()     | Return the type of a given column as a string.                                           |
| SaveGraph()         | Export the computation graph of an RDataFrame in graphviz format for easy inspection.     |
| ...                 | ...                                                                                      |

The advanatage of RDataFrame is that it is compiled and can be used to process large datasets in a very efficient way using multithreading.

Simply put: 

`ROOT::EnableImplicitMT();`


More exmaples on `RDataFrame` can be found in this tutorial [software-carpentry root tutorial](https://github.com/root-project/software-carpentry/)

## Work with the CSV (comma-separated values) Data Source
In this exercise, we will produce a plot of the invariant mass of muon pairs coming from real data of the CMS experiment from proton-proton collisions at 2011([DOI: 10.7483/OPENDATA.CMS.CB8H.MFFA](http://opendata.cern.ch/record/700)).

Let's first create an RDF that will read from the CSV file.

 The types of the columns will be automatically inferred.

In [ ]:
import ROOT

fileName = "../../data/tdf014_CsvDataSource_MuRun2010B.csv"
rdf = ROOT.RDF.FromCSV(fileName)

The kinematic properties of the muons are stored in columns. 

For the "first" muon they are called `Q1, E1, px1, py1, pz1, pt1, eta1, phi1,` and for the "second" one `Q2, E2, px2, py2, pz2, pt2, eta2, phi2`.

In [ ]:
rdf.Display().Print()

* Apply a cut to the entries, selecting the ones that have muons with opposite charge. Charges are stored in the `Q1` and `Q2` columns which has type `Long64_t`

* Now we will apply a first filter based on two columns of the CSV, and we will define a new column that will contain the invariant mass.

* Create a histogram of the invariant mass of the dimuon system. Choose a generous interval, say from 2 to 120 GeV. The formula for the invariant mass can be found [here](https://en.wikipedia.org/wiki/Invariant_mass).

## Exercise: dimuon invariant mass with RDataFrame

**Goal:** starting from a CSV file with muon kinematics, produce and draw the invariant mass spectrum of opposite‑sign dimuons.

### Instructions

1. **Create an RDataFrame from the CSV file**
   - Use the file `../../data/tdf014_CsvDataSource_MuRun2010B.csv`.
   - Use `ROOT.RDF.FromCSV(...)` to create a dataframe called `rdf`.

2. **Select opposite‑sign muon pairs**
   - Filter events so that the product of the charges is negative:
     \[
       Q_1 * Q_2 < 0
     \]
   - Use `Filter` with a **string expression**, e.g.  
     `rdf.Filter("Q1 * Q2 < 0")`.

3. **Define the invariant mass column**
   - Define a new column `m` using
     \[
       m = \sqrt{(E_1 + E_2)^2 - \left[(p_{x1}+p_{x2})^2 + (p_{y1}+p_{y2})^2 + (p_{z1}+p_{z2})^2\right]}
     \]
   - Use `Define("m", "<C++ expression>")` with a **string expression only** (e.g., "sqrt(x)" or "pow(x,n)" ).

4. **Fill and draw the histogram**
   - Create a 1D histogram of `m` with:
     - name `"invMass"`,
     - title like `"CMS Opendata: dimuon mass; m_{μμ} [GeV/c^{2}]; Events"`,
     - 512 bins,
     - x‑range from 2 to 110 GeV.
   - Use `Histo1D((name, title, nbins, xmin, xmax), "m")`.
   - Draw the histogram on a `TCanvas` and:
     - set log‑x and log‑y (`SetLogx`, `SetLogy`),
     - draw the histogram.

### You should use

- `ROOT.RDF.FromCSV(...)`
- `RDataFrame.Filter("<C++ expression>")`
- `RDataFrame.Define("<new_column>", "<C++ expression>")`
- `RDataFrame.Histo1D((name, title, nbins, xmin, xmax), "<column>")`
- `ROOT.TCanvas`, `SetLogx`, `SetLogy`, `Draw`

What resonances are you seeing?